In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Добавляем путь к основной папке проекта, чтобы иметь возможность делать импорт из src
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from collections import defaultdict
from pathlib import Path
import random
import time
from typing import Any, Dict, List

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pytorch_metric_learning import losses
from pytorch_metric_learning.distances import CosineSimilarity
import torch
import torch.nn as nn
import tqdm
import wandb

from src.data.nn_preprocessing import clean_company_name_string
from src.data.lstm_dataloader import CompanyNameDataLoader
from src.models.lstm import LSTMNetwork

In [4]:
train_data = pd.read_csv('../data/processed/train_companies_for_metric_learning.csv')
valid_data = pd.read_csv('../data/processed/valid_companies_for_metric_learning.csv')


# Цикл обучения

In [5]:
contrastive_loss = losses.ContrastiveLoss(pos_margin=1, neg_margin=0, distance=CosineSimilarity())
output_path = Path('../models')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [7]:
lr = 1e-3
emb_dim = 80
hidden_size = 80
num_layers = 3
dropout = 0.0
batch_size = 16
proj_size = 0
exp_name = f'LSTM_train_{emb_dim}_{hidden_size}_{num_layers}_{dropout}_{proj_size}_stop_words'

config = dict (
    learning_rate = lr,
    architecture = "LSTM",
    batch_size = batch_size,
    hidden_size = hidden_size,
    num_layers = num_layers
)

wandb.init(
  project='dl_cnp_23',
  name=exp_name,
  config=config,
)

train_dataloader = CompanyNameDataLoader(train_data, shuffle=True, preprocessing=clean_company_name_string)
valid_dataloader = CompanyNameDataLoader(valid_data, shuffle=False, preprocessing=clean_company_name_string)

net = LSTMNetwork(emb_dim=emb_dim,
                  hidden_size=hidden_size,
                  num_layers=num_layers,
                  dropout=dropout,
                  proj_size=proj_size).to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
best_valid_loss = np.inf

for epoch in range(20):
    print(f'Epoch {epoch}')
    train_losses = []
    for batch in tqdm.tqdm(train_dataloader):
        optimizer.zero_grad()
        pairs = zip(batch[0], batch[1])
        sorted_pair = sorted(pairs, key=lambda x: len(x[0]), reverse=True)
        x, y = zip(*sorted_pair)
        pred = net(x)
        loss = contrastive_loss(pred, torch.Tensor(y).to(device))
        loss.backward()
        train_losses.append(loss.item())
        optimizer.step()
    train_loss = np.mean(train_losses)
    time.sleep(0.5)
    valid_losses = []
    for batch in tqdm.tqdm(valid_dataloader):
        pairs = zip(batch[0], batch[1])
        sorted_pair = sorted(pairs, key=lambda x: len(x[0]), reverse=True)
        x, y = zip(*sorted_pair)
        with torch.no_grad():
            pred = net(x)
            loss = contrastive_loss(pred, torch.Tensor(y).to(device))
        valid_losses.append(loss.item())
    valid_loss = np.mean(valid_losses)
    print(f'train loss: {train_loss}, valid_loss: {valid_loss}')
    wandb.log({'train_loss': train_loss, 'valid_loss': valid_loss}, step=epoch)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(net.state_dict(), output_path / f'{exp_name}_best.pth')
    time.sleep(0.5)

wandb.finish()

wandb: Currently logged in as: mgurevich. Use `wandb login --relogin` to force relogin


Epoch 0


1096it [00:11, 99.13it/s]
107it [00:00, 300.56it/s]


train loss: 0.43472485411504325, valid_loss: 0.35685104075993335
Epoch 1


1096it [00:11, 93.65it/s] 
107it [00:00, 293.35it/s]


...

1096it [00:11, 96.15it/s]
107it [00:00, 298.85it/s]


train loss: 0.0678343432727871, valid_loss: 0.20008771242922016
Epoch 19


1096it [00:11, 95.51it/s]
107it [00:00, 308.11it/s]


train loss: 0.06578497065535625, valid_loss: 0.22071510028294244


train_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
valid_loss,█▅▃▄▃▃▂▄▂▂▁▃▂▃▂▃▁▁▂▃
train_loss,0.06578
valid_loss,0.22072
